In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()
os.chdir('drive/My Drive/Sanket/pdADMM-master/serial_pdADMM/')

In [ ]:
!ls

ADMM_10_layer.py  pdadmm_cifar100_full_batch_1000_iter_1e-4_lr_500_5layers.npz
ADMM_5_layer.py   pdadmm_cifar100_full_batch_100_iter_1e-4_lr_500.npz
GOL		  readme.md
pdADMM		  setup.py


###Reading GTSRB (Greman Traffic Sign Dataset)

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from sklearn.metrics import accuracy_score
np.random.seed(42)

In [ ]:
data_dir = '/content/drive/MyDrive/Sanket/GSTD'
train_path = '/content/drive/MyDrive/Sanket/GSTD/Train'
test_path = '/content/drive/MyDrive/Sanket/GSTD/Test.csv'

# Resizing the images to 32x32x3
IMG_HEIGHT = 32
IMG_WIDTH = 32
channels = 3

In [ ]:
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES

43

In [ ]:
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

(39309, 32, 32, 3) (39309,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_data, image_labels, test_size=0.3, random_state=42, shuffle=True)

#X_train = X_train/255
#X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_test.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_test.shape)

X_train.shape (27516, 32, 32, 3)
X_valid.shape (11793, 32, 32, 3)
y_train.shape (27516,)
y_valid.shape (11793,)


###pdADMM/input_data

In [ ]:
import numpy as np
import sys
import tensorflow as tf
import scipy
#import scipy.io as sio
from keras.utils.np_utils import to_categorical
import tensorflow_datasets as tfds

In [ ]:
class DataSubset(object):
	def __init__(self, xs, ys):
		self.xs = xs
		self.n = xs.shape[0]
		self.ys = ys
		self.batch_start = 0
		self.cur_order = np.random.permutation(self.n)

	def next_batch(self, batch_size, reshuffle_after_pass=True, swapaxes=False):
		if self.n < batch_size:
			raise ValueError('Batch size can be at most the dataset size')
		actual_batch_size = min(batch_size, self.n - self.batch_start)
		if actual_batch_size < batch_size:
			if reshuffle_after_pass:
				self.cur_order = np.random.permutation(self.n)
			self.batch_start = 0
		batch_end = self.batch_start + batch_size
		batch_xs = self.xs[self.cur_order[self.batch_start : batch_end], ...]
		batch_ys = self.ys[self.cur_order[self.batch_start : batch_end], ...]
		self.batch_start += batch_size
		if swapaxes:
			batch_xs = np.swapaxes(batch_xs, 0, 1)
			batch_ys = np.swapaxes(batch_ys, 0, 1)
		return batch_xs, batch_ys

In [ ]:
class gtsrb():
	def __init__(self):
		#self.mnist = tf.keras.datasets.cifar10
		#(self.x_train, self.y_train), (self.x_test, self.y_test) = self.mnist.load_data()
		index = [i in (0, 6) for i in y_train]
		# Uncomment to normalize to (0, 1)
		self.x_train, self.x_test = X_train / 255.0, X_test / 255.0
		self.x_train = self.x_train.reshape(27516, 32 * 32*3)
		self.x_train_down_sample = self.x_train.reshape((27516, 32, 32,3))
		self.x_train_down_sample = scipy.ndimage.zoom(self.x_train_down_sample, (1, 0.5, 0.5,1), order=1).reshape(27516, 16 * 16*3)
		self.y_train = to_categorical(y_train, num_classes=43).reshape(27516, 43)
		self.x_train =self.x_train[index]
		self.x_train_down_sample = self.x_train_down_sample[index]
		self.y_train =self.y_train[index]

		index = [i in (0, 6) for i in y_test]
		self.x_test = self.x_test.reshape(11793,  32* 32 * 3)
		self.x_test_down_sample = self.x_test.reshape((11793, 32, 32,3))
		self.x_test_down_sample = scipy.ndimage.zoom(self.x_test_down_sample, (1, 0.5, 0.5,1), order=1).reshape(11793, 16 * 16*3)
		self.y_test = to_categorical(y_test, num_classes=43).reshape(11793, 43)

		self.x_test =self.x_test[index]
		self.x_test_down_sample = self.x_test_down_sample[index]
		self.y_test =self.y_test[index]

		self.train = DataSubset(self.x_train, self.y_train)
		self.test = DataSubset(self.x_test, self.y_test)
		self.train_down_sample = DataSubset(self.x_train_down_sample, self.y_train)
		self.test_down_sample = DataSubset(self.x_test_down_sample, self.y_test)

###Serial pdADMM

In [ ]:
a = []
b = []
c = []
d = []

In [ ]:
import tensorflow as tf
import numpy as np
import sys
import time
if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")
from pdADMM import common
from pdADMM.input_data import mnist,fashion_mnist,kmnist,svhn,cifar10,cifar100


# initialize the neural network
def Net(images, label, num_of_neurons):
    seed_num = 0
    tf.random.set_seed(seed=seed_num)
    W1 = tf.Variable(tf.random.normal(shape=(num_of_neurons, 16 * 16 * 3),mean=0,stddev=0.1))
    tf.random.set_seed(seed=seed_num)
    b1 = tf.Variable(tf.random.normal(shape=(num_of_neurons, 1),mean=0,stddev=0.1))
    z1 = tf.matmul(W1, images) + b1
    q1 = common.relu(z1)
    p2 = q1
    tf.random.set_seed(seed=seed_num)
    W2 = tf.Variable(tf.random.normal(shape=(num_of_neurons, num_of_neurons),mean=0,stddev=0.1))
    tf.random.set_seed(seed=seed_num)
    b2 = tf.Variable(tf.random.normal(shape=(num_of_neurons, 1),mean=0,stddev=0.1))
    z2 = tf.matmul(W2, q1) + b2
    q2 = common.relu(z2)
    p3 = q2
    tf.random.set_seed(seed=seed_num)
    W3 = tf.Variable(tf.random.normal(shape=(num_of_neurons, num_of_neurons),mean=0,stddev=0.1))
    tf.random.set_seed(seed=seed_num)
    b3 = tf.Variable(tf.random.normal(shape=(num_of_neurons, 1),mean=0,stddev=0.1))
    z3 = tf.matmul(W3, p3) + b3
    q3 = common.relu(z3)
    p4 = q3
    tf.random.set_seed(seed=seed_num)
    W4 = tf.Variable(tf.random.normal(shape=(num_of_neurons, num_of_neurons),mean=0,stddev=0.1))
    tf.random.set_seed(seed=seed_num)
    b4 = tf.Variable(tf.random.normal(shape=(num_of_neurons, 1),mean=0,stddev=0.1))
    z4 = tf.matmul(W4, p4) + b4
    q4 = common.relu(z4)
    p5 = q4
    tf.random.set_seed(seed=seed_num)
    W5 = tf.Variable(tf.random.normal(shape=(43, num_of_neurons),mean=0,stddev=0.1))
    tf.random.set_seed(seed=seed_num)
    b5 = tf.Variable(tf.random.normal(shape=(43, 1),mean=0,stddev=0.1))
    imask = tf.equal(label, 0)
    z5 = tf.where(imask, -tf.ones_like(label), tf.ones_like(label))
    return W1,b1,z1,q1,p2,W2,b2,z2,q2,p3,W3,b3,z3,q3,p4,W4,b4,z4,q4,p5,W5,b5,z5

# return the accuracy of the neural network model
def test_accuracy(W1, b1, W2, b2, W3, b3,W4,b4,W5,b5, images, labels):
    nums = int(labels.shape[1])
    z1 = tf.matmul(W1, images) + b1
    q1 = common.relu(z1)
    p2 = q1
    z2 = tf.matmul(W2, p2) + b2
    q2 = common.relu(z2)
    p3 = q2
    z3 = tf.matmul(W3, p3) + b3
    q3 = common.relu(z3)
    p4 = q3
    z4 = tf.matmul(W4, p4) + b4
    q4 = common.relu(z4)
    p5 = q4
    z5 = tf.matmul(W5, p5) + b5
    cost = common.cross_entropy_with_softmax(labels, z5) / nums
    label = tf.argmax(labels, axis=0)
    pred = tf.argmax(z5, axis=0)
    return (tf.reduce_sum(tf.cast(tf.equal(pred, label),tf.float32)) / nums,cost)

mnist = gtsrb()

print(mnist)
#initialization
x_train = mnist.train_down_sample.xs.astype(np.float32)
y_train = mnist.train_down_sample.ys.astype(np.float32)
x_train = tf.transpose(x_train)
y_train = tf.transpose(y_train)
x_train = tf.convert_to_tensor(x_train,dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train,dtype=tf.float32)
x_test = mnist.test_down_sample.xs.astype(np.float32)
y_test = mnist.test_down_sample.ys.astype(np.float32)
x_test = tf.transpose(x_test)
y_test = tf.transpose(y_test)
x_test = tf.convert_to_tensor(x_test,dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test,dtype=tf.float32)
num_of_neurons = 500
ITER = 50
index = 0
W1, b1, z1, q1, p2, W2, b2, z2, q2, p3, W3, b3, z3, q3, p4, W4, b4, z4, q4, p5, W5, b5, z5= Net(x_train, y_train, num_of_neurons)
u1=tf.zeros(q1.shape)
u2=tf.zeros(q2.shape)
u3=tf.zeros(q3.shape)
u4=tf.zeros(q4.shape)
train_acc=np.zeros(ITER)
test_acc=np.zeros(ITER)
linear_r=np.zeros(ITER)
objective_value=np.zeros(ITER)
train_cost=np.zeros(ITER)
test_cost=np.zeros(ITER)
rho = 1e-4
for i in range(ITER):
    pre = time.time()
    print("iter=",i)
    p2 =common.update_p(p2,q1,W2,b2,z2,u1,rho)
    p3 =common.update_p(p3,q2,W3,b3,z3,u2,rho)
    p4 =common.update_p(p4,q3,W4,b4,z4,u3,rho)
    p5 =common.update_p(p5,q4,W5,b5,z5,u4,rho)
    W1 =common.update_W(x_train,b1,z1,W1,rho)
    W2 =common.update_W(p2,b2,z2,W2,rho)
    W3 =common.update_W(p3,b3,z3,W3,rho)
    W4 =common.update_W(p4,b4,z4,W4,rho)
    W5 =common.update_W(p5,b5,z5,W5,rho)
    b1 =common.update_b(x_train,W1,z1,b1,rho)
    b2 =common.update_b(p2,W2,z2,b2,rho)
    b3 =common.update_b(p3,W3,z3,b3,rho)
    b4 =common.update_b(p4,W4,z4,b4,rho)
    b5 =common.update_b(p5,W5,z5,b5,rho)
    z1 =common.update_z(z1,x_train,W1,b1,q1,rho)
    z2 =common.update_z(z2,p2,W2,b2,q2,rho)
    z3 =common.update_z(z3,p3,W3,b3,q3,rho)
    z4 =common.update_z(z4,p4,W4,b4,q4,rho)
    z5 =common.update_zl(p5,W5,b5,y_train,z5,rho)
    q1 =common.update_q(p2,z1,u1,rho)
    q2 =common.update_q(p3,z2,u2,rho)
    q3 =common.update_q(p4,z3,u3,rho)
    q4 =common.update_q(p5,z4,u4,rho)
    u1 = u1 +rho*(p2-q1)
    u2 = u2 +rho*(p3-q2)
    u3 = u3 +rho*(p4-q3)
    u4 = u4 +rho*(p5-q4)
    print("Time per iteration:", time.time() - pre)
    print("rho=",rho)
    (train_acc[i],train_cost[i])=test_accuracy(W1, b1, W2, b2, W3, b3,W4, b4, W5, b5, x_train,y_train)
    print("training cost:", train_cost[i])
    print("training acc:",train_acc[i])
    a.append(train_acc[i])
    c.append(train_cost[i])
    (test_acc[i],test_cost[i])=test_accuracy(W1, b1, W2, b2, W3, b3,W4, b4, W5, b5, x_test, y_test)
    print("test cost:", test_cost[i])
    print("test acc:",test_acc[i])
    b.append(test_acc[i])
    d.append(test_cost[i])
np.savez('pdadmm_gtsrb_full_batch_1000_iter_1e-4_lr_'+repr(num_of_neurons)+'_5layers', W1, b1, z1, q1, p2, W2, b2, z2, q2, p3, W3, b3, z3, q3, p4, W4, b4, z4, q4, p5, W5, b5, z5)

iter= 0
Time per iteration: 1.4241437911987305
rho= 0.0001
training cost: 4.9186177253723145
training acc: 0.08350730687379837
test cost: 5.011746883392334
test acc: 0.08074533939361572
iter= 1
Time per iteration: 0.5748298168182373
rho= 0.0001
training cost: 1.3089076280593872
training acc: 0.699373722076416
test cost: 1.4224083423614502
test acc: 0.6708074808120728
iter= 2
Time per iteration: 0.21454977989196777
rho= 0.0001
training cost: 0.8367424607276917
training acc: 0.7390396595001221
test cost: 0.9067503213882446
test acc: 0.7142857313156128
iter= 3
Time per iteration: 0.22191667556762695
rho= 0.0001
training cost: 0.5144783854484558
training acc: 0.8162839412689209
test cost: 0.5592697858810425
test acc: 0.8136646151542664
iter= 4
Time per iteration: 0.2181096076965332
rho= 0.0001
training cost: 0.3567388951778412
training acc: 0.8622129559516907
test cost: 0.3924347460269928
test acc: 0.8447204828262329
iter= 5
Time per iteration: 0.23057341575622559
rho= 0.0001
training cost

In [ ]:
a

[0.08350730687379837,
 0.699373722076416,
 0.7390396595001221,
 0.8162839412689209,
 0.8622129559516907,
 0.9039666056632996,
 0.9290187954902649,
 0.9331941604614258,
 0.9373695254325867,
 0.9394571781158447,
 0.9394571781158447,
 0.9436325430870056,
 0.9457202553749084,
 0.9540709853172302,
 0.9561586380004883,
 0.9561586380004883,
 0.9582463502883911,
 0.9645093679428101,
 0.9665970802307129,
 0.9728600978851318,
 0.9770354628562927,
 0.9770354628562927,
 0.9770354628562927,
 0.9770354628562927,
 0.9770354628562927,
 0.9791231751441956,
 0.9812108278274536,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9853861927986145,
 0.9874739050865173,
 0.9874739050865173,
 0.9874739050865173,
 0.9895615577697754,
 0.9895615577697754,
 0.9895615577697754,
 0.9916492700576782,
 0.9916492700576782,
 0.9916492700576782,
 0.9916492700576782,
 0.9916492700576782,
 0.9916492700

In [ ]:
train_acc = pd.DataFrame (a, columns = ['dlADMM'])
test_acc = pd.DataFrame (b, columns = ['dlADMM'])
train_loss = pd.DataFrame (c, columns = ['dlADMM'])
test_loss = pd.DataFrame (d, columns = ['dlADMM'])

In [ ]:
train_acc.to_csv('/content/drive/MyDrive/Sreenitha/Projects/Metrocad_ext/opt_graph/dlADMM/train_acc.csv', index=False)
test_acc.to_csv('/content/drive/MyDrive/Sreenitha/Projects/Metrocad_ext/opt_graph/dlADMM/test_acc.csv', index=False)
train_loss.to_csv('/content/drive/MyDrive/Sreenitha/Projects/Metrocad_ext/opt_graph/dlADMM/train_loss.csv', index=False)
test_loss.to_csv('/content/drive/MyDrive/Sreenitha/Projects/Metrocad_ext/opt_graph/dlADMM/test_loss.csv', index=False)